In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import functions.process_data

In [5]:
import pandas as pd 
import pandas_profiling as pdp
import numpy as np
import pprint as pp
import seaborn as sns
from matplotlib import pyplot as plt
from catboost import Pool, CatBoostRegressor

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import cross_val_score


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [6]:
files = get_kaggle_data(
    competition="house-prices-advanced-regression-techniques"
    )

files downloaded for house-prices-advanced-regression-techniques competition
   test.csv
   data_description.txt
   train.csv
   sample_submission.csv
located in /competition_data/


In [7]:
input_data = import_to_df(files)

What files did we get?

In [8]:
input_data.keys()

dict_keys(['test', 'train', 'sample_submission'])

In [9]:
input_train_df = input_data['train']
input_test_df = input_data['test']

In [16]:
train_df = process_data(input_train_df)
train_df = calculate_inflation(
    data_with_price=train_df,
    data_to_append_to=train_df,
    group_by_cat=['OverallQual'],
    group_by_band=['GrLivArea'],
    year='YrSold',
    price='SalePrice'
)
target = 'SalePrice'
id_row = 'Id'
train_predictors = [
    c for c in train_df.columns 
    if target not in c
    if id_row not in c
]

/Users/oliverharvey/anaconda3/envs/titanic/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/Users/oliverharvey/anaconda3/envs/titanic/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [17]:
test_df = process_data(input_test_df)
test_df = calculate_inflation(
    data_with_price=train_df,
    data_to_append_to=test_df,
    group_by_cat=['OverallQual'],
    group_by_band=['GrLivArea'],
    year='YrSold',
    price='SalePrice'
)
test_predictors = [
    c for c in test_df.columns 
    if target not in c
    if id_row not in c
]

/Users/oliverharvey/anaconda3/envs/titanic/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/Users/oliverharvey/anaconda3/envs/titanic/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Only keep columns in both dfs. Needed in case one hots generate unique columns

In [18]:
cols_to_use = list(set(train_predictors).intersection(test_predictors))

Also create smaller feature set with decently correlated variables

In [19]:
corr = train_df.corr()

In [20]:
corr_target = abs(corr['SalePrice'])
# corr_target.sort_values(ascending=False)
relevant_features = corr_target[corr_target>0.3].index.tolist()

In [21]:
# relevant_features
cols_to_use_small = list(set(cols_to_use).intersection(relevant_features))

In [22]:
len(cols_to_use), len(train_predictors)

(305, 318)

Removed 13 columns

In [23]:
X_train = train_df[cols_to_use]
y_train = train_df[target]

X_test = test_df[cols_to_use]
y_test_labels = test_df[id_row]

In [24]:
X_train_small = train_df[cols_to_use_small]
X_test_small = test_df[cols_to_use_small]

In [26]:
# initialize Pool
train_pool = Pool(X_train, y_train)
test_pool = Pool(X_test) 

param_dist = {
    'depth': [6,8,10],
    'learning_rate' : [0.01, 0.05, 0.1],
    'iterations'    : [30, 50, 100],
    'l2_leaf_reg': [1, 3, 5, 7, 9]
}
#Instantiate RandomSearchCV object
cbr = CatBoostRegressor()

random_grid = RandomizedSearchCV(cbr, param_dist, cv =5)

#Fit the model
random_grid.fit(X_train, y_train, verbose=2)

# Print the tuned parameters and score
print(random_grid.best_params_)
print(random_grid.best_score_)


remaining: 288ms
90:	learn: 47228.6920437	total: 2.38s	remaining: 235ms
92:	learn: 46712.8714966	total: 2.45s	remaining: 185ms
94:	learn: 46203.0500683	total: 2.61s	remaining: 137ms
96:	learn: 45705.2196857	total: 2.67s	remaining: 82.7ms
98:	learn: 45235.7813907	total: 2.74s	remaining: 27.7ms
99:	learn: 45012.7920712	total: 2.77s	remaining: 0us
0:	learn: 66456.5046958	total: 17.9ms	remaining: 1.77s
2:	learn: 65518.8040523	total: 54.6ms	remaining: 1.77s
4:	learn: 64651.3165986	total: 98.2ms	remaining: 1.86s
6:	learn: 63738.2224662	total: 157ms	remaining: 2.08s
8:	learn: 62863.2828398	total: 216ms	remaining: 2.18s
10:	learn: 62022.5148996	total: 269ms	remaining: 2.17s
12:	learn: 61183.2068635	total: 324ms	remaining: 2.17s
14:	learn: 60384.5852684	total: 377ms	remaining: 2.14s
16:	learn: 59586.6242868	total: 432ms	remaining: 2.11s
18:	learn: 58820.3721452	total: 478ms	remaining: 2.04s
20:	learn: 58041.6704741	total: 519ms	remaining: 1.95s
22:	learn: 57299.2694060	total: 561ms	remaining: 1

Use these params to start grid search

In [25]:
param_grid = {
    'learning_rate': [0.05, 0.1, 0.2, 0.4],
    'iterations': [80, 100, 150, 200], 
    'depth': [ 6, 8, 10],
    'l2_leaf_reg': [1, 3, 5, 9],
    'random_strength': [1,2]
}

grid_search = GridSearchCV(estimator = cbr, param_grid = param_grid, 
                          cv = 4, n_jobs = -1, verbose = 1)
grid_search.fit(X_train, y_train)

NameError: name 'cbr' is not defined

In [212]:
print(grid_search.best_params_)
print(grid_search.best_score_)

{'depth': 4, 'iterations': 150, 'l2_leaf_reg': 1, 'learning_rate': 0.1, 'random_strength': 1}
0.8789847410809853


In [285]:
grid_search_small = GridSearchCV(estimator = cbr, param_grid = param_grid, 
                          cv = 4, n_jobs = -1, verbose = 1)
grid_search_small.fit(X_train_small, y_train)

Fitting 4 folds for each of 360 candidates, totalling 1440 fits
0:	learn: 68320.5155412	total: 52.4ms	remaining: 4.14s
1:	learn: 60241.0297156	total: 106ms	remaining: 4.12s
2:	learn: 53919.3858586	total: 150ms	remaining: 3.86s
3:	learn: 48283.6112880	total: 195ms	remaining: 3.7s
4:	learn: 44236.6117771	total: 227ms	remaining: 3.41s
5:	learn: 40926.4679700	total: 268ms	remaining: 3.31s
6:	learn: 38278.9689674	total: 308ms	remaining: 3.21s
7:	learn: 35747.0529618	total: 349ms	remaining: 3.14s
8:	learn: 33640.1432039	total: 390ms	remaining: 3.08s
9:	learn: 32137.3551152	total: 426ms	remaining: 2.98s
10:	learn: 30461.6872348	total: 460ms	remaining: 2.89s
11:	learn: 29163.9706023	total: 499ms	remaining: 2.82s
12:	learn: 27832.7083433	total: 538ms	remaining: 2.77s
13:	learn: 26800.4544658	total: 575ms	remaining: 2.71s
14:	learn: 25949.2357129	total: 614ms	remaining: 2.66s
15:	learn: 25328.5371871	total: 646ms	remaining: 2.58s
16:	learn: 24752.0459579	total: 681ms	remaining: 2.52s
17:	learn: 

GridSearchCV(cv=4,
             estimator=<catboost.core.CatBoostRegressor object at 0x7fb96c0812b0>,
             n_jobs=-1,
             param_grid={'depth': [4, 6, 10], 'iterations': [50, 80, 100, 150],
                         'l2_leaf_reg': [1, 3, 5],
                         'learning_rate': [0.05, 0.1, 0.2, 0.4, 0.8],
                         'random_strength': [1, 2]},
             verbose=1)

In [286]:
print(grid_search_small.best_params_)
print(grid_search_small.best_score_)

{'depth': 6, 'iterations': 80, 'l2_leaf_reg': 3, 'learning_rate': 0.2, 'random_strength': 1}
0.8614599373271481


In [213]:
best_cb = CatBoostRegressor(**grid_search.best_params_)

In [214]:
manual_cb = CatBoostRegressor(**{'depth': 10, 'iterations': 200, 'l2_leaf_reg': 5, 'learning_rate': 0.4, 'random_strength': 2})

## Random Forest

First use random search again

In [120]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 40]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 5, 10, 50]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {
    'n_estimators': n_estimators,
    'max_features': max_features,
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
    'bootstrap': bootstrap
}

In [121]:
rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4, 5, 10,
                                                             50],
                                        'min_samples_split': [2, 5, 10, 40],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [123]:
display(rf_random.best_params_)
display(rf_random.best_score_)

{'n_estimators': 1200,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 110,
 'bootstrap': False}

0.8644008815472926

In [175]:
best_rf_rand = RandomForestRegressor(**rf_random.best_params_)

Use these parameters to create full grid search

In [215]:
rf_param_grid = {
    'bootstrap': [False],
    'max_depth': [ 100, 110, 120],
    'max_features': ['sqrt'],
    'min_samples_leaf': [ 1, 10, 50],
    'min_samples_split': [ 2, 6, 12, 18],
    'n_estimators': [ 1000, 1200, 1400]
}

rf_grid_search = GridSearchCV(estimator = rf, param_grid = rf_param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2)
rf_grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


GridSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'bootstrap': [False], 'max_depth': [100, 110, 120],
                         'max_features': ['sqrt'],
                         'min_samples_leaf': [1, 10, 50],
                         'min_samples_split': [2, 6, 12, 18],
                         'n_estimators': [1000, 1200, 1400]},
             verbose=2)

In [216]:
display(rf_grid_search.best_params_)
display(rf_grid_search.best_score_)

{'bootstrap': False,
 'max_depth': 100,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 1200}

0.8604519890431638

In [287]:
rf_grid_search_small = GridSearchCV(estimator = rf, param_grid = rf_param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2)
rf_grid_search_small.fit(X_train_small, y_train)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


GridSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'bootstrap': [False], 'max_depth': [100, 110, 120],
                         'max_features': ['sqrt'],
                         'min_samples_leaf': [1, 10, 50],
                         'min_samples_split': [2, 6, 12, 18],
                         'n_estimators': [1000, 1200, 1400]},
             verbose=2)

In [288]:
display(rf_grid_search_small.best_params_)
display(rf_grid_search_small.best_score_)

{'bootstrap': False,
 'max_depth': 120,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 1400}

0.8646889087423275

In [217]:
best_rf = RandomForestRegressor(**rf_grid_search.best_params_)

## Elastic Search
Try a linear model for rounding off ensembling. Apparently this is good for many features not enough obs

In [289]:
en_grid = {
    'alpha': [1e-5, 1e-3, 0.0, 10.0, 100.0],
    'l1_ratio': [0,0.1, 0.3, 0.5, 1, 2]
}
en = ElasticNet()
en_grid_search = GridSearchCV(estimator = en, param_grid = en_grid, 
                          cv = 5, n_jobs = -1, verbose = 2)
en_grid_search.fit(X_train_small, y_train)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
/Users/oliverharvey/anaconda3/envs/titanic/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [0.80296783 0.80296527 0.80296011 0.8029549  0.80294163        nan
 0.80384405 0.80380894 0.80372137 0.80360102 0.80294174        nan
 0.80294163 0.80294163 0.80294163 0.80294163 0.80294163        nan
 0.73171771 0.73304775 0.73652086 0.74177239 0.80368416        nan
 0.7193208  0.71938918 0.71966137 0.72033568 0.80535976        nan]
  warnings.warn(
/Users/oliverharvey/anaconda3/envs/titanic/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 237804370874.63586, tolerance: 920791133.4609975
  model = cd_fast.enet_coordinate_descent(


GridSearchCV(cv=5, estimator=ElasticNet(), n_jobs=-1,
             param_grid={'alpha': [1e-05, 0.001, 0.0, 10.0, 100.0],
                         'l1_ratio': [0, 0.1, 0.3, 0.5, 1, 2]},
             verbose=2)

In [290]:
display(en_grid_search.best_params_)
display(en_grid_search.best_score_)

{'alpha': 100.0, 'l1_ratio': 1}

0.8053597604488232

In [177]:
best_en = ElasticNet(**en_grid_search.best_params_)
best_en

ElasticNet(alpha=100.0, l1_ratio=1)

Ensemble them all together

In [294]:
def top_n_models(search_results, n, model, prefix):
    results = pd.DataFrame(search_results.cv_results_)
    results.sort_values(by='rank_test_score', inplace=True)
    top_n = results.loc[results.index[:5], 'params'].to_list()
    models = []
    for i, param in enumerate(top_n):
        label = f'{prefix}_{i}'
        model_instance = model(**param)
        models.append((label, model_instance))
    return models

catboost_5 = top_n_models(grid_search, 2, CatBoostRegressor, "Catboost_grid")
catboost_small_5 = top_n_models(grid_search_small, 1, CatBoostRegressor, "Catboost_small_grid")
rf_5 = top_n_models(rf_grid_search, 1, RandomForestRegressor, "RF_grid")
rf_small_5 = top_n_models(rf_grid_search_small, 1, RandomForestRegressor, "RF_small_grid")

In [302]:
# cv = KFold(n_splits=5, random_state=1, shuffle=True)

ensemble = VotingRegressor(
    estimators=[
        *catboost_5,
        *rf_5,
        *catboost_small_5,
        # *rf_small_5,
        # ('ElasticNet', best_en),
        # ('RF Grid', best_rf),
        # ('RF Random', best_rf_rand),
        # ('Grad Boost', best_cb),
        # ('My catboost', manual_cb)
    ])
# scores_best_ens = cross_val_score(ensemble, X_train, y_train, cv=cv, n_jobs=-1)
# scores_best_ens

In [303]:
ensemble.fit(X_train, y_train)

n: 14252.6871975	total: 598ms	remaining: 281ms
68:	learn: 14168.4838216	total: 608ms	remaining: 273ms
69:	learn: 14093.1807157	total: 617ms	remaining: 265ms
70:	learn: 13971.0958502	total: 627ms	remaining: 256ms
71:	learn: 13843.9640791	total: 637ms	remaining: 248ms
72:	learn: 13705.5346125	total: 647ms	remaining: 239ms
73:	learn: 13576.5771494	total: 657ms	remaining: 231ms
74:	learn: 13515.2333327	total: 665ms	remaining: 222ms
75:	learn: 13430.1493269	total: 676ms	remaining: 214ms
76:	learn: 13310.8064332	total: 686ms	remaining: 205ms
77:	learn: 13216.5320479	total: 694ms	remaining: 196ms
78:	learn: 13106.3753496	total: 702ms	remaining: 187ms
79:	learn: 13034.4092712	total: 709ms	remaining: 177ms
80:	learn: 12896.9274627	total: 716ms	remaining: 168ms
81:	learn: 12788.0802086	total: 724ms	remaining: 159ms
82:	learn: 12700.5412606	total: 731ms	remaining: 150ms
83:	learn: 12584.0005357	total: 740ms	remaining: 141ms
84:	learn: 12454.3014641	total: 747ms	remaining: 132ms
85:	learn: 12382.5

VotingRegressor(estimators=[('Catboost_grid_0',
                             <catboost.core.CatBoostRegressor object at 0x7fb9ceeb10a0>),
                            ('Catboost_grid_1',
                             <catboost.core.CatBoostRegressor object at 0x7fb9ceeb1f40>),
                            ('Catboost_grid_2',
                             <catboost.core.CatBoostRegressor object at 0x7fb9d28bfbe0>),
                            ('Catboost_grid_3',
                             <catboost.core.CatBoostRegressor object at 0x7fb9d28bff10>),
                            ('Catboost_grid_4'...
                             <catboost.core.CatBoostRegressor object at 0x7fb9ceeb1580>),
                            ('Catboost_small_grid_1',
                             <catboost.core.CatBoostRegressor object at 0x7fb9ceeb1460>),
                            ('Catboost_small_grid_2',
                             <catboost.core.CatBoostRegressor object at 0x7fb9ceeb1220>),
                    

# make preds and submit

In [304]:
y_pred = ensemble.predict(X_test)

submission_df = pd.DataFrame()
submission_df[id_row] = y_test_labels
submission_df[target] = y_pred

In [298]:
submission_df.head()

,Id,SalePrice
0,1461,124260.892478
1,1469,183318.720394
2,1470,126755.855889
3,1472,98566.403669
4,1473,101133.980599


In [233]:
input_data['sample_submission'].head()

,Id,SalePrice
0,1461,169277.052498
1,1462,187758.393989
2,1463,183583.683570
3,1464,179317.477511
4,1465,150730.079977


In [305]:
submission_df.to_csv('submission10.csv', index=False)

In [306]:
!kaggle competitions submit -f 'submission10.csv' -m 'top ns ensemble with small versions v2' house-prices-advanced-regression-techniques

100%|██████████████████████████████████████| 33.7k/33.7k [00:01<00:00, 17.7kB/s]
Successfully submitted to House Prices - Advanced Regression Techniques

In [307]:
!kaggle competitions submissions -c house-prices-advanced-regression-techniques

fileName          date                 description                             status    publicScore  privateScore  
----------------  -------------------  --------------------------------------  --------  -----------  ------------  
submission10.csv  2021-05-07 16:05:25  top ns ensemble with small versions v2  complete  0.13050      None          
submission9.csv   2021-05-07 15:57:13  top ns ensemble with small versions     complete  0.13287      None          
submission8.csv   2021-05-06 07:28:46  top ns ensemble                         complete  0.13273      None          
submission7.csv   2021-05-06 07:24:29  top 5s ensemble                         complete  0.13314      None          
submission6.csv   2021-05-06 06:54:29  ensemble 4                              complete  0.13241      None          
submission6.csv   2021-05-06 06:54:04  ensemble 4                              complete  0.13241      None          
submission5.csv   2021-05-06 06:51:41  ensemble 3               